<a id="top"></a>
# Tutorial Title

***

## Learning Goals
Write three to five learning goals. A learning goal should describe what a reader should know or be able to do by the end of the tutorial that they didn't know or couldn't do before.

```
By the end of this tutorial, you will:

- Use astroquery to access the HST and Gaia archives
- Understand the drizzle technique
- Run a source extractor on HST images.
- Cross match the HST and Gaia sources.
- Create your view own previews of targets for future proposals, including for JWST!

```

## Introduction
Write a short introduction explaining the purpose of the tutorial. Define any terms or common acronyms that your audience may not know. If you're using some kind of domain-specific astronomical symbol or unusual mathematical concept, make sure you define it (for example, in its mathematical form) and link to any definitions (from literature, Wikipedia, etc.).

If there are background materials or resources that may be useful to the reader to provide additional context, you may link to it here. If your tutorial is a continuation from another tutorial, or there are other tutorials that would be useful for the reader to read before or after your tutorial, mention that here as well.

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

If any of the below modules are not currently installed, get them with pip3 in your command line:
`pip3 install -u stwcs astropy astroquery`

In [34]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Mast
from astroquery.mast import Observations
import os, shutil, stwcs, subprocess, glob 

Let's start by retrieving images of our object of interest from the HST Archive. For this project, we will consider the Orion Nebula. Astroquery.Observations allows you to query targets by name, and it is helpful to additionally specify a search radius. We download the products and store them locally. 

More famous objects have been observed by Hubble many times, so the download may be prohibitively large. You may want to select only images in a certain band, or apply filters on any of the [fields listed here.]( https://mast.stsci.edu/api/v0/_c_a_o_mfields.html)

In [39]:
obs_table = Observations.query_criteria(target_name=["Orion","OrionNebula"],intentType='science',dataproduct_type='image')
obs_table

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
str5,int64,str4,str38,str11,float64,float64,float64,float64,float64,str8,str26,float64,float64,str23,str27,float64,str7,str20,str5,str10,str10,int64,str10,str133,str88,str87,str6,bool,float64,str7,str8,str9
image,3,HST,j8up02usq,ORIONNEBULA,83.81875,-5.38975,53025.179451655094,53025.1811996875,150.0,Optical,CLEAR1L;FR656N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.78548604,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410508 83.798123 -5.388606 83.819603 -5.36935 83.839412 -5.391251 83.817931 -5.410508,mast:HST/product/j8up02usq_drz.jpg,mast:HST/product/j8up02usq_drz.fits,PUBLIC,False,nan,science,24016393,121741157
image,3,HST,j8up02utq,ORIONNEBULA,83.81875,-5.38975,53025.18352569445,53025.18527376158,150.0,Optical,CLEAR1L;FR656N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.78619211,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410508 83.798123 -5.388606 83.819603 -5.36935 83.839412 -5.391251 83.817931 -5.410508,mast:HST/product/j8up02utq_drz.jpg,mast:HST/product/j8up02utq_drz.fits,PUBLIC,False,nan,science,24016394,121741219
image,3,HST,j8up02uuq,ORIONNEBULA,83.81875,-5.38975,53025.18759980324,53025.18934783565,150.0,Optical,CLEAR1L;FR656N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.78692124,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410508 83.798123 -5.388606 83.819603 -5.36935 83.839412 -5.391251 83.817931 -5.410508,mast:HST/product/j8up02uuq_drz.jpg,mast:HST/product/j8up02uuq_drz.fits,PUBLIC,False,nan,science,24016395,121741204
image,3,HST,j8up02uvq,ORIONNEBULA,83.81875,-5.38975,53025.19178943287,53025.193537465275,150.0,Optical,CLEAR1L;FR388N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.78775462,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410508 83.798123 -5.388606 83.819603 -5.369349 83.839412 -5.391251 83.817931 -5.410508,mast:HST/product/j8up02uvq_drz.jpg,mast:HST/product/j8up02uvq_drz.fits,PUBLIC,False,nan,science,24016396,121741214
image,3,HST,j8up02uwq,ORIONNEBULA,83.81875,-5.38975,53025.19584054398,53025.20048190972,400.0,Optical,CLEAR1L;FR388N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.60020823,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410508 83.798123 -5.388605 83.819603 -5.369349 83.839412 -5.391251 83.817931 -5.410508,mast:HST/product/j8up02uwq_drz.jpg,mast:HST/product/j8up02uwq_drz.fits,PUBLIC,False,nan,science,24016397,121741185
image,3,HST,j8up02uyq,ORIONNEBULA,83.81875,-5.38975,53025.202993136576,53025.20474116898,150.0,Optical,CLEAR1L;FR505N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.60090273,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410507 83.798123 -5.388605 83.819603 -5.369349 83.839412 -5.391251 83.817931 -5.410507,mast:HST/product/j8up02uyq_drz.jpg,mast:HST/product/j8up02uyq_drz.fits,PUBLIC,False,nan,science,24016398,121741174
image,3,HST,j8up02v0q,ORIONNEBULA,83.81875,-5.38975,53025.20706721065,53025.20881524306,150.0,Optical,CLEAR1L;FR505N,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.78953694,ACS/WFC,"Jogee, Shardha",10057,CAL/ACS,HST,--,CALACS,POLYGON 83.817931 -5.410507 83.798123 -5.388605 83.819603 -5.369349 83.839412 -5.391251 83.817931 -5.410507,mast:HST/product/j8up02v0q_drz.jpg,mast:HST/product/j8up02v0q_drz.fits,PUBLIC,False,nan,science,24016399,121741211
image,3,HST,j8up02v5q,ORIONNEBULA,83.81875,-5.38975,53025.24283128472,53025.24457931713,150.0,Optical,CLEAR1L;FR459M,200.0,1100.0,CALIBRATION,ACS Ramp Filter Calibration,53025.79164349,ACS/WFC,"Jogee, Shardha",10057,CA

In [40]:
download_tab = Observations.download_products(obs_table['obsid'])
download_tab

Local Path,Status,Message,URL
str102,str8,object,object
./mastDownload/HST/j8up02usq/j8up02usq_flt_hlet.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usj_jif.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usj_jit.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_spt.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_trl.fits,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_drz.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_raw.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_flt.jpg,COMPLETE,None,None
./mastDownload/HST/j8up02usq/j8up02usq_raw.fits,COMPLETE,None,None


In [47]:
# for file in download_tab['Local Path']:
#     os.rename(file, os.path.basename(file))
    
# shutil.rmtree('mastDownload')
# !mkdir FITS images
# !mv *fits FITS/
# !mv *jpg images/
!ls

FITS
hst_10057_02_acs_wfc_fr388n_j8up02uv_trl.txt
hst_10057_02_acs_wfc_fr388n_j8up02uw_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v5_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v7_trl.txt
hst_10057_02_acs_wfc_fr459m_j8up02v9_trl.txt
hst_10057_02_acs_wfc_fr505n_j8up02uy_trl.txt
hst_10057_02_acs_wfc_fr505n_j8up02v0_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02us_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02ut_trl.txt
hst_10057_02_acs_wfc_fr656n_j8up02uu_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vb_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vc_trl.txt
hst_10057_02_acs_wfc_fr914m_j8up02vd_trl.txt
images
previewing_transient_candidates.ipynb


You will notice that the number of downloaded files far exceeds the number of observations collected in `obs_table`. To understand them, let us look into the one of the accompanying `.txt` files.

In [56]:
with open('hst_10057_02_acs_wfc_fr388n_j8up02uv_trl.txt') as f:
    a = f.readlines()
    for line in a[:10]:
        print(line.split('\n')[0])

2021330044237 INFO src=drizzlepac.hapsequencer- Run start time: 2021-11-26 04:42:37.777383
2021330044237 INFO src=drizzlepac.hapsequencer- Parse the poller and determine what exposures need to be combined into separate products.

2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02usq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02utq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uuq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uvq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uwq_flt.fits found in current working directory.
2021330044237 INFO src=drizzlepac.haputils.poller_utils- Input image j8up02uyq_flt.fits found in current working directory.
2021330044237 INFO src=dri

This tells us that several files, j...fits, were collected at each observation and combined into one image per filter using `drizzlepac`. In fact, we can use `drizzlepac` to combine all of these images into a single composite image, deeper than any individual one.

The Drizzle technique is a way of orienting all the images into exactly the same reference frame. This is crucial since the telescope shifts its pointing direction slightly all the time; in addition, different instruments on the telescope point in different directions. To align the images, we first need to find all the point sources in them, and then feed in the point source lists, along with the WCS coordinate information, to `drizzlepac`.

## Assignment 1: 
Pull all available observations for the galaxy cluster Abell 2744, also known as Pandora's Cluster. Try your favourite object. If you query only by target name, do you get too many results? What filters might help you narrow down to the data most useful to your proposal? Besides images, what data types might serve you? 

In [ ]:
obs_table = Observations.query_object( )


At this point, all the FITS files live in the FITS directory, while pre-processed images are in images/. Take a moment to look through the images in your preferred image viewer. Next, we will load the FITS files and manipulate them.

In [35]:
files = glob.glob('FITS/*fits')
files.sort() 
files

['FITS/hlsp_orion_hst_acs_colorimage_b_v1_drz.fits',
 'FITS/hlsp_orion_hst_acs_colorimage_g_v1_drz.fits',
 'FITS/hlsp_orion_hst_acs_colorimage_r_v1_drz.fits',
 'FITS/x22q0101t_c0f.fits',
 'FITS/x22q0101t_c1f.fits',
 'FITS/x22q0101t_d0f.fits',
 'FITS/x22q0101t_dgr.fits',
 'FITS/x22q0101t_q0f.fits',
 'FITS/x22q0101t_shf.fits',
 'FITS/x22q0101t_trl.fits',
 'FITS/x22q0101t_ulf.fits',
 'FITS/x22q0201t_c0f.fits',
 'FITS/x22q0201t_c1f.fits',
 'FITS/x22q0201t_d0f.fits',
 'FITS/x22q0201t_dgr.fits',
 'FITS/x22q0201t_q0f.fits',
 'FITS/x22q0201t_shf.fits',
 'FITS/x22q0201t_trl.fits',
 'FITS/x22q0201t_ulf.fits']

In [ ]:
What are all these x..fits files?

***

## Main Content

The main content of your tutorial should be subdivided into numbered sections with useful, descriptive headings that make sense based on the content. Break sections up with standard Markdown syntax headings:

```
## Section 1

Intro to Section 1

### Subsection 1a

More detailed info about Section 1

## Section 2

A complete thought that's as important as Section 1 but doesn't need subsections.

```

### Loading Data

Loading data and file information should appear within your main content, at the same time the data is going to be used, if possible. These elements of your tutorial can be their own sections within the main content, but avoid generic or vague headings like “Loading Data” and instead use descriptive headings pertinent to the content of the tutorial and the actual data being downloaded or files being used.

If the user needs to download data to run the tutorial properly, where possible, use [Astroquery](https://astroquery.readthedocs.io/en/latest/) (or similar) to retrieve files. If this is not possible, see the [data guide](https://github.com/spacetelescope/style-guides/blob/master/guides/where-to-put-your-data.md) for other options.

For example, if we wanted to query for data from MAST for Kepler we might do something like:

In [ ]:
keplerObs = Observations.query_criteria(target_name='kplr008957091', obs_collection='Kepler')
keplerProds = Observations.get_product_list(keplerObs[0])
yourProd = Observations.filter_products(keplerProds,extension='kplr008957091-2012277125453_lpd-targ.fits.gz',
                                        mrp_only=False)


### File Information

Explain pertinent details about the file you've just downloaded. For example, if working with Kepler light curves, explain what's in the different file extensions:

```
- No. 0 (Primary): This HDU contains metadata related to the entire file.
- No. 1 (Light curve): This HDU contains a binary table that holds data like
  flux measurements and times. We will extract information from here when we
  define the parameters for the light curve plot.
- No. 2 (Aperture): This HDU contains the image extension with data collected
  from the aperture. We will also use this to display a bitmask plot that
  visually represents the optimal aperture used to create the SAP_FLUX column in
  HDU1.

```

Where possible (if the code supports it), use code examples that visually display the data in the tutorial. For example, if you are showing an object such as a Table, display a preview of the table:

In [15]:
yourProd[0:5]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str6,str10,str36,str59,str1,str110,str7,str28,str1,str1,str6,str1,str7,str44,int64,str10
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Preview-Full,C,mast:Kepler/url/missions/kepler/previews/0089/008957091/kplr008957091-2013098041711_llc_bw_large.png,PREVIEW,--,--,--,Kepler,--,GO30032,kplr008957091-2013098041711_llc_bw_large.png,29653,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q10,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2011271113734_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2011271113734_llc.fits,486720,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q11,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012004120508_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012004120508_llc.fits,506880,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q12,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012088054726_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012088054726_llc.fits,434880,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q13,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012179063303_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012179063303_llc.fits,472320,9000159893


In [ ]:
# Download the products
Observations.download_products(yourProd, mrp_only=False, cache=False)

## Exercises
Exercises are optional, but encouraged. Exercises can be woven into the main content of your tutorial, or appear in their own section toward the end of the tutorial. Final exercises can be more challenging, similar to homework problems. They can be minimal or take as long as 30 minutes to an hour to complete. If you do have one or more exercises in your tutorial, be sure to leave a blank code cell underneath each to show the reader that they're meant to try out their new skill right there. You may also want to include a "solutions" notebook next to your main notebook for the reader to check their work after they have finished their attempt.

## Additional Resources

This section is optional. Try to weave resource links into the main content of your tutorial so that they are falling in line with the context of your writing. For resources that do not fit cleanly into your narrative, you may include an additional resources section at the end of your tutorial. Usually a list of links using Markdown bullet list plus link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/)

## About this Notebook
Let the world know who the author of this great tutorial is! If possible and appropriate, include a contact email address for users who might need support (for example, `archive@stsci.edu`). You can also optionally include keywords, your funding source, or a last update date in this section.

**Author:** Jessie Blogs, Archive Scientist.  
**Updated On:** YYYY-MM-DD

## Citations
Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

```
If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

```

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 